In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

import kraft

In [ ]:
point_x_dimension = np.asarray(
    (
        (0, 0),
        (1, 0),
        (2, 0),
        (0, 1),
        (1, 1),
        (2, 1),
        (0, 2),
        (1, 2),
        (2, 2),
        (0.5, 1.75),
        (2, 3),
    )
)

point_xs, point_ys = point_x_dimension.T

In [ ]:
def plot(point_xs, point_ys, line_xs, line_ys):

    layouts_size = 640

    kraft.plot_plotly(
        {
            "layout": {"height": layouts_size, "width": layouts_size,},
            "data": [
                {"x": point_xs, "y": point_ys, "mode": "markers",},
                {"x": line_xs, "y": line_ys, "mode": "lines", "opacity": 0.32,},
            ],
        }
    )

## get_triangulation

In [ ]:
triangulation_xs, triangulation_ys = kraft.get_triangulation(point_x_dimension)

plot(point_xs, point_ys, triangulation_xs, triangulation_ys)

## get_convex_hull

In [ ]:
convex_hull_xs, convex_hull_ys = kraft.get_convex_hull(point_x_dimension)

plot(point_xs, point_ys, convex_hull_xs, convex_hull_ys)

## map_points_by_pull

## map_points

## plot_gps_map

## GPSMap

In [ ]:
point_x_node = pd.read_csv("w.tsv", sep="\t", index_col=0)

point_x_node.columns.name = "Node"

In [ ]:
node_x_node_distance = squareform(pdist(point_x_node.values.T))

gps_map = kraft.GPSMap(node_x_node_distance, point_x_node)

In [ ]:
gps_map.plot()

In [ ]:
points_to_highlight = ["Feature 6", "Feature 10", "Feature 29"]

print(gps_map.point_x_dimension.loc[points_to_highlight])

gps_map.plot(points_to_highlight=points_to_highlight)

In [ ]:
point_value = gps_map.point_x_dimension.sum(axis=1)

point_value.name = "Dimension Sum"

gps_map.plot(point_value=point_value)

point_value[::2] = np.nan

gps_map.plot(point_value=point_value)

In [ ]:
gps_map.set_point_label(
    pd.Series(
        gps_map.point_x_dimension.values.argmax(axis=1),
        index=point_x_node.index,
        name="Point Label",
    )
)

gps_map.plot()

In [ ]:
gps_map.plot(point_value=point_value)

In [ ]:
gps_map.predict(
    gps_map.point_x_node.loc[points_to_highlight], point_trace={"marker": {"size": 24}},
)

In [ ]:
h = pd.read_csv("kras_gps_map.h.tsv", sep="\t", index_col=0)

h.columns.name = "Cell Line"

h_ = np.apply_along_axis(kraft.normalize, 1, h.values, "-0-").clip(min=-3, max=3)

h_ = np.apply_along_axis(kraft.normalize, 1, h_, "0-1")

h_ **= 2

h = pd.DataFrame(h_, index=h.index, columns=h.columns)

node_x_node_distance = squareform(pdist(h.values))

gps_map = kraft.GPSMap(node_x_node_distance, h.T)

gps_map.set_point_label(
    pd.read_csv("kras_gps_map.k_x_h_element.tsv", sep="\t", index_col=0).loc["K15"]
)

gps_map.plot()

In [ ]:
h_ = h.copy()

h_.columns.name = "New {}".format(h.columns.name)

gps_map.predict(h_.T)

## write_gps_map

In [ ]:
gps_map_path = os.path.expanduser("~/gps_map.pickle.gz")

kraft.write_gps_map(gps_map_path, gps_map)

## read_gps_map

In [ ]:
gps_map = kraft.read_gps_map(gps_map_path)

gps_map.plot()